In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 15.8 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install albumentations==0.4.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65153 sha256=412d455142b595607552477056dff1bc1e0cfb44157bc3d6d7a6c9977925c3c8
  Stored in directory: /root/.cache/pip/wheels/f9/d7/0c/6ed42fd872f7d1af78b25045f8b16be330f2c70ae72c83e37d
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.14
    Uninstalling albumentations-1.4.14:
      Successfully uninstalled albumentations-1.4.14


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO
import os
import cv2



In [4]:
# Data Augmentation Pipeline
def get_augmentations():
    return A.Compose([
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),  # Replaces ColorJitter
        A.ToGray(p=0.3),
        A.RandomGamma(p=0.3),
        A.GaussianBlur(blur_limit=7, p=0.2),
        A.MotionBlur(blur_limit=7, p=0.2),
        A.MedianBlur(blur_limit=7, p=0.2),
        A.ISONoise(p=0.2),
        A.ImageCompression(quality_lower=60, p=0.3),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.5),
        A.RandomRain(p=0.3, blur_value=3, brightness_coefficient=0.9),
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, p=0.3),
        A.RandomSnow(p=0.3, snow_point_lower=0.1, snow_point_upper=0.3),
        A.Resize(640, 640),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])




In [5]:
# Dataset class
class CombinedDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.label_paths = []

        # Process the nested Fog directory
        condition_path = os.path.join(root_dir, 'Fog', 'Fog')
        label_path = os.path.join(condition_path, 'Fog_YOLO_darknet')  # Path to label files

        print(f"Processing condition: Fog")
        print(f"Images directory: {condition_path}")
        print(f"Labels directory: {label_path}")

        if os.path.isdir(condition_path):
            # Iterate over the image files in the nested Fog folder
            for img_file in os.listdir(condition_path):
                if img_file.endswith('.jpg') or img_file.endswith('.png'):
                    img_full_path = os.path.join(condition_path, img_file)
                    self.image_paths.append(img_full_path)

                    # Generate the corresponding label filename
                    label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
                    full_label_path = os.path.join(label_path, label_file)

                    # Check if the label file exists
                    if os.path.exists(full_label_path):
                        self.label_paths.append(full_label_path)
                    else:
                        print(f"Label file not found: {full_label_path}")

        print(f"Total images found: {len(self.image_paths)}")
        print(f"Total labels found: {len(self.label_paths)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load label
        with open(label_path, 'r') as f:
            labels = f.readlines()

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        # Process labels to YOLO format
        processed_labels = []
        for label in labels:
            label = list(map(float, label.strip().split()))
            processed_labels.append(torch.tensor(label))

        return image, torch.stack(processed_labels)



In [6]:

# Transformer Model for Image Enhancement
class AllWeatherEnhancementTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(input_dim, 3 * 640 * 640)  # Output is 3 channels (RGB)

    def forward(self, x):
        x = self.transformer(x)
        x = self.fc(x)
        return x



In [7]:
# Training function for enhancement model
def train_enhancement_network(model, dataloader, num_epochs, device):
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        for batch in dataloader:
            inputs, _ = batch
            inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, inputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")



In [8]:
# Fine-tuning function for YOLOv8
def fine_tune_yolov8(enhancement_model, yolo_model, dataloader, num_epochs, device):
    optimizer = torch.optim.Adam(list(enhancement_model.parameters()) + list(yolo_model.parameters()))

    for epoch in range(num_epochs):
        for batch in dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            # Enhance images
            enhanced_inputs = enhancement_model(inputs)

            # YOLOv8 forward pass and loss computation
            loss = yolo_model(enhanced_inputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")



In [9]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    data_dir = '/kaggle/input/dawn-low-visibilty-car-detection/DAWN'
    
    # Dataset and Dataloader with augmentation
    combined_dataset = CombinedDataset(data_dir, transform=get_augmentations())
    
    # Debug: Print the number of samples in the dataset
    print(f"Number of samples in the dataset: {len(combined_dataset)}")
    
    # If dataset length is 0, check file paths and structure.
    if len(combined_dataset) == 0:
        raise ValueError("The dataset is empty. Please check the dataset paths or contents.")
    
    # Set pin_memory to True for faster data transfer to GPU
    dataloader = DataLoader(combined_dataset, batch_size=16, shuffle=True, pin_memory=True)
    
    # Initialize models
    enhancement_model = AllWeatherEnhancementTransformer(input_dim=3 * 512 * 512, num_heads=8, num_layers=4).to(device)
    yolo_model = YOLO('yolov8n.pt').to(device)  # Load pre-trained YOLOv8 model and move it to GPU
    
    # Train enhancement network
    train_enhancement_network(enhancement_model, dataloader, num_epochs=50, device=device)
    
    # Fine-tune YOLOv8 with enhancement network
    fine_tune_yolov8(enhancement_model, yolo_model, dataloader, num_epochs=50, device=device)
    
    # Save the trained models
    torch.save(enhancement_model.state_dict(), "enhancement_model.pth")
    yolo_model.save("fine_tuned_yolov8.pt")


In [ ]:
if __name__ == "__main__":
    main()


Processing condition: Fog
Images directory: /kaggle/input/dawn-low-visibilty-car-detection/DAWN/Fog/Fog
Labels directory: /kaggle/input/dawn-low-visibilty-car-detection/DAWN/Fog/Fog/Fog_YOLO_darknet
Total images found: 300
Total labels found: 300
Number of samples in the dataset: 300
